<a href="https://colab.research.google.com/github/XmYx/ainodes-pyside/blob/main/aiNodes_webAPI_colab_v0_0_2_public.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown **NVIDIA GPU**
import subprocess, os, sys
sub_p_res = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total,memory.free', '--format=csv,noheader'], stdout=subprocess.PIPE).stdout.decode('utf-8')
print(f"{sub_p_res[:-1]}")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import subprocess, time, gc, os, sys

def setup_environment():
    print_subprocess = False
    use_xformers_for_colab = True
    try:
        ipy = get_ipython()
    except:
        ipy = 'could not get_ipython'
    if 'google.colab' in str(ipy):
        print("..setting up environment")
        start_time = time.time()
        all_process = [
            ['pip', 'install', 'torch==1.12.1+cu113', 'torchvision==0.13.1+cu113', '--extra-index-url', 'https://download.pytorch.org/whl/cu113'],
            ['pip', 'install', 'omegaconf==2.2.3', 'einops==0.4.1', 'pytorch-lightning==1.7.4', 'torchmetrics==0.9.3', 'torchtext==0.13.1', 'transformers==4.21.2', 'kornia==0.6.7'],
            ['git', 'clone', 'https://github.com/deforum-art/deforum-stable-diffusion'],
            ['pip', 'install', 'accelerate', 'ftfy', 'jsonmerge', 'matplotlib', 'resize-right', 'timm', 'torchdiffeq','scikit-learn'],
            ['pip', 'install', 'fastapi', 'uvicorn', 'pydantic'],
            ['git', 'clone', 'https://github.com/XmYx/ainodes-pyside'],
        ]
        for process in all_process:
            running = subprocess.run(process,stdout=subprocess.PIPE).stdout.decode('utf-8')
            if print_subprocess:
                print(running)
        with open('deforum-stable-diffusion/src/k_diffusion/__init__.py', 'w') as f:
            f.write('')
        sys.path.extend([
            'deforum-stable-diffusion/',
            'deforum-stable-diffusion/src',
        ])
        end_time = time.time()

        if use_xformers_for_colab:

            print("..installing xformers")

            all_process = [['pip', 'install', 'triton==2.0.0.dev20220701']]
            for process in all_process:
                running = subprocess.run(process,stdout=subprocess.PIPE).stdout.decode('utf-8')
                if print_subprocess:
                    print(running)
                    
            v_card_name = subprocess.run(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'], stdout=subprocess.PIPE).stdout.decode('utf-8')
            if 't4' in v_card_name.lower():
                name_to_download = 'T4'
            elif 'v100' in v_card_name.lower():
                name_to_download = 'V100'
            elif 'a100' in v_card_name.lower():
                name_to_download = 'A100'
            elif 'p100' in v_card_name.lower():
                name_to_download = 'P100'
            else:
                print(v_card_name + ' is currently not supported with xformers flash attention in deforum!')

            x_ver = 'xformers-0.0.13.dev0-py3-none-any.whl'
            x_link = 'https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/' + name_to_download + '/' + x_ver
        
            all_process = [
                ['wget', x_link],
                ['pip', 'install', x_ver],
                ['mv', 'deforum-stable-diffusion/src/ldm/modules/attention.py', 'deforum-stable-diffusion/src/ldm/modules/attention_backup.py'],
                ['mv', 'deforum-stable-diffusion/src/ldm/modules/attention_xformers.py', 'deforum-stable-diffusion/src/ldm/modules/attention.py']
            ]

            for process in all_process:
                running = subprocess.run(process,stdout=subprocess.PIPE).stdout.decode('utf-8')
                if print_subprocess:
                    print(running)

            print(f"Environment set up in {end_time-start_time:.0f} seconds")
    else:
        sys.path.extend([
            'src'
        ])
    return

setup_environment()

import torch
import random
import clip
#from IPython import display
#from types import SimpleNamespace
#from helpers.save_images import get_output_folder
#from helpers.settings import load_args
#from helpers.render import render_animation, render_input_video, render_image_batch, render_interpolation
#from helpers.model_load import make_linear_decode, load_model, get_model_output_paths
#from helpers.aesthetics import load_aesthetics_model

#@markdown **Setup and Start**

#def Root():
#    models_path = "models" #@param {type:"string"}
#    configs_path = "configs" #@param {type:"string"}
#    output_path = "output" #@param {type:"string"}
#    mount_google_drive = True #@param {type:"boolean"}
#    models_path_gdrive = "/content/drive/MyDrive/AI/models" #@param {type:"string"}
#    output_path_gdrive = "/content/drive/MyDrive/AI/StableDiffusion" #@param {type:"string"}
#
#    #@markdown **Model Setup**
#    model_config = "v1-inference.yaml" #@param ["custom","v1-inference.yaml"]
#    model_checkpoint =  "v1-5-pruned-emaonly.ckpt" #@param ["custom","v1-5-pruned.ckpt","v1-5-pruned-emaonly.ckpt","sd-v1-4-full-ema.ckpt","sd-v1-4.ckpt","sd-v1-3-full-ema.ckpt","sd-v1-3.ckpt","sd-v1-2-full-ema.ckpt","sd-v1-2.ckpt","sd-v1-1-full-ema.ckpt","sd-v1-1.ckpt", "robo-diffusion-v1.ckpt","wd-v1-3-float16.ckpt"]
#    custom_config_path = "" #@param {type:"string"}
#    custom_checkpoint_path = "" #@param {type:"string"}
#    half_precision = True
#    return locals()

#root = Root()
#root = SimpleNamespace(**root)

#root.models_path, root.output_path = get_model_output_paths(root)
#root.model, root.device = load_model(root, 
#                                    load_on_run_all=True
#                                    , 
#                                    check_sha256=True
#                                    )

#!pip install -e git+https://github.com/osi1880vr/pytorch3d-lite.git
!pip install fonts
!pip install font-roboto piexif lark
!pip install nest_asyncio
!pip install pyngrok

!pip install k-diffusion
!pip install -qq https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.14/xformers-0.0.14.dev0-cp37-cp37m-linux_x86_64.whl
%cd /content/ainodes-pyside
!python install.py
!pip install -e .
!git clone https://github.com/osi1880vr/AdaBins.git
!git clone https://github.com/osi1880vr/MiDaS.git
!git clone https://github.com/osi1880vr/pytorch3d-lite.git
!pip install -e pytorch3d-lite
!pip install -e MiDaS
!pip install -e AdaBins
%cd /content/ainodes-pyside
!git pull
!python apis/fast_app.py

#Start Again

In [ ]:
#@markdown **Start Again**

%cd /content/ainodes-pyside
!git checkout main
!git pull
!python apis/fast_app.py